In [1]:
import time
from lib import bitMapFile, boardToBitMap, display_board, bitMapToBoard
import chess
import itertools
from tqdm.notebook import tqdm as tqdm
import json
from time import sleep

In [37]:
def mapHash(position, oppBitMap):
    return (oppBitMap << 6 | position)

In [40]:
finalBitMap = {}
totalValues = 0

for position in tqdm(range(64)):
    board = chess.Board(None)
    board.set_piece_at( position, chess.Piece(chess.BISHOP, chess.WHITE))
    legal_moves = [
        move for move in board.legal_moves if move.from_square == position]
    powerset = []
    for r in range(len(legal_moves) + 1):
        for subset in itertools.combinations(legal_moves, r):
            powerset.append(subset)
    for moveSet in powerset:
        tempBoard = chess.Board(None)
        tempBoard.set_piece_at(
            position, chess.Piece(chess.BISHOP, chess.WHITE))
        oppBitMap = 0
        for move in moveSet:
            tempBoard.set_piece_at(
                move.to_square, chess.Piece(chess.PAWN, chess.BLACK))
            oppBitMap |= 1 << (63-move.to_square)
        tempLegalMoves = [
            move for move in tempBoard.legal_moves if move.from_square == position]
        bitMap = 0
        for move in tempLegalMoves:
            bitMap |= 1 << (63-move.to_square)
        currHash = mapHash(position, oppBitMap)
        assert currHash not in finalBitMap, "Hash Collision Found!"
        finalBitMap[currHash] = bitMap
        totalValues += 1

  0%|          | 0/64 [00:00<?, ?it/s]

In [41]:
with open('diagonalMap.json', 'w+') as f:
    json.dump(finalBitMap, f)

In [11]:
board = bitMapToBoard(bitMapFile(
    "/home/mystic/Programming/mystic-bot/src/sample/default.json"))
board.fen()

'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'

In [4]:
bitMap

{'rooks': 2535301201046759118051251060736,
 'knights': 20480,
 'bishops': 649037107316853453847787017863168,
 'queens': 42535295865117307932921825928971026432,
 'kings': 9444732965743585394688,
 'pawns': 1486775696367240623243722359808,
 'metadata': 65792,
 'latest_move': 0}